In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join("../"))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import os.path
import random

import get_params as params
import torch
import torch.nn as nn
import torch.optim as optim
from classifier_models.utils import return_model_accuracy
from classifier_models.vpnn import (FCMVPCSNN, FCMVPSNN, FCTMVPSNN, FCVPNN,
                                    FCVPSNN, FCVPTSNN)
from dataloaders.load_dataset import load_ecg_real
from nn_layers.vp_layers import *


In [3]:

params.get('../network_configs/ECG_VPNN.yaml')

torch.manual_seed(params.SEED_NUM)
random.seed(params.SEED_NUM)

data_path = os.path.join('../data', 'ecg', 'mitbih')
train_loader, test_loader = load_ecg_real(params.batch_size, data_path)
number_of_observations = len(train_loader.dataset)


In [4]:
if params.classifier_name == 'FCVPNN':
    '''FCVPNN - setup'''
    classifier = FCVPNN(params.vp_latent_dim, params.num_classes,
                        device=params.device, penalty=params.L2penalty, init_vp=params.init_vp)
elif params.classifier_name == 'FCVPSNN':
    '''FCVPSNN - setup'''
    init_vp = [0.1, 0]
    classifier = FCVPSNN(params.vp_latent_dim, params.num_classes, device=params.device,
                         penalty=params.L2penalty, init_vp=init_vp, n_steps=params.n_steps)
elif params.classifier_name == 'FCVPTSNN':
    '''FCVPTSNN - setup'''
    init_vp = [0.001, 0.001]
    classifier = FCVPTSNN(params.vp_latent_dim, params.num_classes, device=params.device,
                          penalty=params.L2penalty, init_vp=init_vp, n_steps=params.n_steps)
elif params.classifier_name == 'FCMVPSNN':
    '''FCMVPSNN - setup'''
    init_mvp = [random.random() for i in range(2 * params.n_steps)]
    classifier = FCMVPSNN(params.vp_latent_dim, params.num_classes, device=params.device,
                          penalty=params.L2penalty, init_vp=init_mvp, n_steps=params.n_steps, m_vp=params.n_steps)
elif params.classifier_name == 'FCTMVPSNN':
    '''FCTMVPSNN - setup'''
    init_mvp = [random.random()/2 for i in range(2 * params.n_steps)]
    classifier = FCTMVPSNN(params.vp_latent_dim, params.num_classes, device=params.device,
                           penalty=params.L2penalty, init_vp=init_mvp, n_steps=params.n_steps, m_vp=params.n_steps)
elif params.classifier_name == 'FCMVPCSNN':
    '''FCMVPCSNN - setup'''
    init_mvp = [random.random() for i in range(2 * params.n_steps)]
    # insert some weights which prooved to be suitable for ECG classification
    init_mvp[0:2] = [0.1, 0.0]
    classifier = FCMVPCSNN(params.vp_latent_dim, params.num_classes, device=params.device,
                           penalty=params.L2penalty, init_vp=init_mvp, n_steps=params.n_steps, m_vp=params.n_steps)

classifier.to(params.device)


FCVPTSNN(
  (tdvp): temporal_tdvp_layer()
  (tdlinear1): tdLinear(
    in_features=4, out_features=8, bias=True
    (spike): LIFSpike()
  )
  (tdlinear2): tdLinear(
    in_features=8, out_features=20, bias=False
    (spike): LIFSpike()
  )
  (tdlinear3): tdLinear(
    in_features=20, out_features=2, bias=False
    (spike): LIFSpike()
  )
  (softmax): Softmax(dim=-1)
  (ReLU): ReLU()
  (membrane_output_layer): MembraneOutputLayer()
)

In [5]:
# define objective and optimizer functions
criterion = nn.BCELoss()
optimizer = optim.Adagrad(classifier.parameters(), 
                       lr=params.lr,
                       lr_decay=params.lr_decay,
                       weight_decay=0.001)


In [6]:
# loop over the dataset multiple times
for epoch in range(params.epochs_cls):  
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(params.device), data[1].to(params.device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = classifier(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    # print epoch performance
    with torch.no_grad():
        train_acc = return_model_accuracy(classifier, train_loader, params.device)
        test_acc = return_model_accuracy(classifier, test_loader, params.device)
        print(f'[Epoch: {epoch + 1}], Train accuracy: {train_acc:.2f}%, Test accuracy: {test_acc:.2f}%')
        print(f'[Epoch: {epoch + 1}], Train accuracy: {train_acc:.2f}%, Test accuracy: {test_acc:.2f}%', file=open('output.txt', 'a'))

print('Finished Training')

[Epoch: 1], Train accuracy: 58.72%, Test accuracy: 63.90%
[Epoch: 2], Train accuracy: 65.38%, Test accuracy: 64.24%
[Epoch: 3], Train accuracy: 67.72%, Test accuracy: 65.03%
[Epoch: 4], Train accuracy: 71.08%, Test accuracy: 77.07%
[Epoch: 5], Train accuracy: 79.71%, Test accuracy: 84.80%
[Epoch: 6], Train accuracy: 79.94%, Test accuracy: 86.01%
[Epoch: 7], Train accuracy: 80.82%, Test accuracy: 86.61%
[Epoch: 8], Train accuracy: 81.27%, Test accuracy: 86.83%
[Epoch: 9], Train accuracy: 81.37%, Test accuracy: 87.13%
[Epoch: 10], Train accuracy: 81.67%, Test accuracy: 87.42%
[Epoch: 11], Train accuracy: 81.70%, Test accuracy: 87.80%
[Epoch: 12], Train accuracy: 81.87%, Test accuracy: 88.23%
[Epoch: 13], Train accuracy: 82.14%, Test accuracy: 88.45%
[Epoch: 14], Train accuracy: 82.15%, Test accuracy: 88.80%
[Epoch: 15], Train accuracy: 82.32%, Test accuracy: 89.15%
[Epoch: 16], Train accuracy: 82.30%, Test accuracy: 89.61%
[Epoch: 17], Train accuracy: 82.38%, Test accuracy: 89.80%
[Epoch